In [45]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.messages import HumanMessage, AIMessage, ToolMessage
from langgraph.checkpoint.memory import InMemorySaver
from langchain.tools import tool
import asyncio
LLM_CONFIG = {
    # 替换为你的 API Key
    "api_key": "sk-PJfHRIM2Q0K8vXajt7pBjrXFhxbVP3GSsqw6KItdSM2MvOf4",
    # 替换为你的 API 接入点 (Base URL)
    # 例如 DeepSeek: 'https://api.deepseek.com'
    "base_url": "https://api.moonshot.cn/v1",
    # 替换为你需要使用的模型名称
    "model_name": "kimi-k2-turbo-preview",
}
SYSTEM_PROMPT="你是一个智能的助理,你能够根据用户的问题来调用llm。快速的回答用户。最后回答要包含完整文案，要总结之前的所有信息,不要只说生成好了。"

llm=ChatOpenAI(
            model=LLM_CONFIG["model_name"],
            api_key=LLM_CONFIG["api_key"],
            base_url=LLM_CONFIG["base_url"],
            temperature=0.0,
        )



In [46]:
from dashscope.audio.asr import *
from dashscope import MultiModalConversation
import dashscope
import asyncio
import os
import sounddevice as sd
import numpy as np
import threading
import time

import json
import base64
from typing import List, Optional


class Callback(RecognitionCallback):
    def __init__(self, text, frames, owner) -> None:
        self.temp_text = []
        self.text = text
        self.frames = frames
        self.owner = owner

    def on_open(self) -> None:
        print("RecognitionCallback open.")

    def on_close(self) -> None:
        print("RecognitionCallback close.")
        if self.temp_text:
            self.text.extend(self.temp_text)
            self.temp_text.clear()

    def on_complete(self) -> None:
        print("RecognitionCallback completed.")  # translation completed
        if self.temp_text:
            self.text.extend(self.temp_text)
            self.temp_text.clear()

    def on_error(self, message) -> None:
        print("RecognitionCallback task_id: ", message.request_id)
        print("RecognitionCallback error: ", message.message)
        if self.temp_text:
            self.text.extend(self.temp_text)
            self.temp_text.clear()

    def on_event(self, result: RecognitionResult) -> None:
        sentence = result.get_sentence()
        if sentence is None:
            return

        if isinstance(sentence, list):
            sentence_list = sentence
        else:
            sentence_list = [sentence]

        for current in sentence_list:
            if "text" in current:
                print("RecognitionCallback text: ", current["text"])
                self.temp_text.append(current["text"])
                if RecognitionResult.is_sentence_end(current):
                    print(
                        "RecognitionCallback sentence end, request_id:%s, usage:%s"
                        % (result.get_request_id(), result.get_usage(current))
                    )
                    last_text = current.get("text", "")
                    if last_text:
                        self.text.append(last_text)
                    self.temp_text.clear()
            if "emo_tag" in current:
                print("RecognitionCallback emo_tag: ", current["emo_tag"])
                self.owner.emo_tag = current["emo_tag"]

    def reset(self) -> None:
        self.temp_text.clear()


class SpeechService:
    def __init__(self):
        sample_rate = 16000

        # 资源存储变量
        self.frames: List[np.ndarray] = []  # 声音数组
        self.transcribed_text: List[str] = []  # 文本数组
        self.emo_tag = None
        
        # 初始化dashscope
        self.dashscope_api_key = "sk-629432e4c14148f0a28b49d761cdc6b9"
        dashscope.api_key = self.dashscope_api_key
        # 看了官方的api,目前就是支持用callback来进行流式调用,也可以不填这个参数一次性传递完，我选择阻塞的方法因为我的最终转换的text是prompt,这个prompt必须完整
        self.callback = Callback(self.transcribed_text, self.frames, self)
        self._recognition_started = False
        self.SpeechToTextClient = Recognition(
            model="paraformer-realtime-v2",
            format="pcm",
            sample_rate=sample_rate,
            heartbeat=True,
            callback=self.callback,
        )
        try:
            self.SpeechToTextClient.start()  # 流式的开始识别
            self._recognition_started = True
        except Exception as e:
            print(f"Error initializing recognition: {e}")
            self._recognition_started = False
        self.SpeakerClient = MultiModalConversation
        self.is_recording = False

        self.recording_thread = None

        # Audio recording parameters
        self.sample_rate = sample_rate
        self.channels = 1
        self.dtype = np.int16

    def start_recording(self):
        """多线程开启录音"""
        if self.is_recording:
            return False

        if not self._recognition_started:
            try:
                self.SpeechToTextClient.start()
                self._recognition_started = True
            except Exception as e:
                print(f"Error starting recognition: {e}")
                return False

        if self.callback:
            self.callback.reset()

        self.is_recording = True
        self.frames.clear()
        self.transcribed_text.clear()
        self.emo_tag = None

        try:
            # 录音的线程
            self.recording_thread = threading.Thread(target=self._record_audio)
            self.recording_thread.start()
            return True
        except Exception as e:
            print(f"Error starting recording: {e}")
            self.is_recording = False
            if self._recognition_started:
                try:
                    self.SpeechToTextClient.stop()
                except Exception as stop_error:
                    print(f"Error stopping recognition after failure: {stop_error}")
                self._recognition_started = False
            return False

    def stop_recording(self):
        """把暂停的逻辑放到回调函数了,这个保留为主动暂停的函数，可能用不上的"""
        if not self.is_recording:
            return None

        self.is_recording = False

        if self.recording_thread:
            self.recording_thread.join()
            self.recording_thread = None

        if self._recognition_started:
            try:
                self.SpeechToTextClient.stop()
            except Exception as e:
                print(f"Error stopping recognition: {e}")
            self._recognition_started = False
        if not self.transcribed_text:
            return None
        return "".join(self.transcribed_text)

    def _record_audio(self):
        """Record audio using sounddevice"""
        try:
            # Record audio until stopped
            with sd.InputStream(
                samplerate=self.sample_rate,
                channels=self.channels,
                dtype=self.dtype,
                callback=self._audio_callback,
            ) as stream:
                # 当这个结束的时候就没有结束了
                while self.is_recording:
                    sd.sleep(100)  # Sleep in milliseconds
        except Exception as e:
            print(f"Error recording audio: {e}")

    def _audio_callback(self, indata, frames, time_info, status):
        """这是核心的回调函数,sounddevice每录制到一个CHUNK,就会调用一次这个函数。"""
        if not self.is_recording:
            return

        chunk = indata.copy()
        self.frames.append(chunk)

        try:
            self.SpeechToTextClient.send_audio_frame(chunk.tobytes())
        except Exception as e:
            print(f"Error sending audio frame: {e}")
            self.is_recording = False
            raise sd.CallbackStop

    def record_and_transcribe(self, duration=5):
        """Record for specified duration and transcribe(这个只有可能是被api调用了,暂时用不上)"""
        if self.start_recording():
            time.sleep(duration)
            return self.stop_recording()
        return None

    def _speak_blocking(self, text: str) -> None:
        """阻塞式的语音播放逻辑，供线程池调用。"""
        voice = self.SpeakerClient.call(
            model="qwen3-tts-flash",
            api_key=self.dashscope_api_key,
            text=text,
            voice="Cherry",
            language_type="Chinese",
            stream=True,
        )
        try:
            with sd.OutputStream(samplerate=24000, channels=1, dtype="int16") as stream:
                print("音频流已开启，准备播放...")
                for chunk in voice:
                    audio = chunk.output.audio
                    if audio.data is not None:
                        wav_bytes = base64.b64decode(audio.data)
                        audio_np = np.frombuffer(wav_bytes, dtype=np.int16)
                        stream.write(audio_np)
                    if chunk.output.finish_reason == "stop":
                        print(
                            f"服务端标记结束，到期时间: {chunk.output.audio.expires_at}"
                        )
                print("音频数据接收完毕，等待播放完成...")
                time.sleep(0.8)
        except Exception as e:
            print(f"播放时发生错误: {e}")

        print("播放结束，音频流已自动关闭。")

    async def speak(self, text: str) -> None:
        """异步地播放语音，避免阻塞事件循环。"""
        # asyncio.to_thread 适合用来包裹“原本是同步/阻塞”的逻辑
        await asyncio.to_thread(self._speak_blocking, text)

    def cleanup(self):
        """Clean up audio resources"""
        if self.is_recording:
            self.stop_recording()
        if self._recognition_started:
            try:
                self.SpeechToTextClient.stop()
            except Exception as e:
                print(f"Error stopping recognition during cleanup: {e}")
            self._recognition_started = False
        self.frames.clear()

    @property
    def text(self) -> Optional[str]:
        if not self.transcribed_text:
            return None
        combined_text = "".join(self.transcribed_text)
        return combined_text

    @property
    def emotion(self) -> Optional[str]:
        return self.emo_tag

    def clear_text(self):
        self.transcribed_text.clear()


    print("开始测试SpeechService类...")

# 创建SpeechService实例
service = SpeechService()


          


开始测试SpeechService类...
RecognitionCallback open.


In [47]:
"""
Xiaohongshu Marketing Tools
小红书夸张营销文案工具：为大模型提供爆款口吻、标题和文案模板。
"""

from typing import List, Optional



@tool(return_direct=True)
def xhs_style_guide(
    persona: str = "疯批反差萌",
    emoji_density: int = 3,
    safety_note: bool = True,
) -> str:
    """
    返回一份夸张小红书写作指南，直接贴给大模型即可套用。

    Args:
        persona: 口吻人设，如“疯批反差萌”“专业又尖叫”
        emoji_density: 每段建议插入的 emoji 数量
        safety_note: 是否提醒规避医疗/功效绝对化表述

    Returns:
        一段包含语气、结构、标点、标签用法的写作速查表
    """
    guide = [
        f"人设：{persona}，抓马到位但保持真情实感；对读者称呼用“姐妹们/宝子们”。",
        f"语气：开头必须惊叫 + 反复感叹；多用大写和拉长词（太！！！好！！！哭了！！！）。",
        "结构：爆点开头 -> 个人崩溃瞬间/反转 -> 3-5 个细节卖点 -> 强制安利 + 行动口号。",
        f"标点：感叹号连发，省略号制造悬念；每段插入约 {emoji_density} 个 emoji（⚡️🤯😭✨🫶🔥）。",
        "标签：结尾叠加 6-10 个话题标签，包含产品、场景、情绪、趋势关键词。",
    ]
    if safety_note:
        guide.append("合规：避免“治愈/百分百”之类绝对功效词，可用“离谱好用”“像开挂”。")

    return "\n".join(f"- {line}" for line in guide)


@tool(return_direct=True)
def xhs_title_pack(
    product: str,
    target_user: str = "姐妹们",
    scene: Optional[str] = None,
) -> str:
    """
    生成一组高点击小红书风格标题，直接可用。

    Args:
        product: 产品/服务名称
        target_user: 主要受众称呼
        scene: 使用场景或痛点

    Returns:
        6-8 条标题候选，带 emoji 和话题位
    """
    scene_part = f"{scene} " if scene else ""
    titles: List[str] = [
        f"{target_user}崩溃尖叫！{scene_part}{product}真的离谱好用！！！⚡️🤯",
        f"跪了！{product}=开挂神器？我试完沉默了😭",
        f"【别再错过】{scene_part}{product}这波我必须全网喊！！！🔥",
        f"没有对比没有伤害，{product}把我拿捏了…🫠🫶",
        f"年度心动榜第一名：{product}！把状态拉满的一天✨",
        f"冲！{product} = 我最勇敢的一次入手，结果直接上头😳",
        f"反转了！原来{scene_part}{product}才是隐藏王者？！🤯",
    ]
    hashtags = [
        f"#{product}",
        f"# {scene}" if scene else "",
        "# 必入好物",
        "# 尖叫推荐",
        "# 小众宝藏",
    ]
    hashtag_line = " ".join(tag for tag in hashtags if tag)
    titles.append(f"标签备选：{hashtag_line}")
    return "\n".join(f"{idx+1}. {title}" for idx, title in enumerate(titles))


@tool(return_direct=True)
def xhs_hype_copy(
    product: str,
    selling_points: str,
    audience: str = "姐妹们",
    scenario: str = "日常通勤",
    call_to_action: str = "冲！马上安排！",
) -> str:
    """
    生成一篇夸张的小红书种草文案，含开头爆点、细节卖点和标签。

    Args:
        product: 产品/服务名称
        selling_points: 卖点列表，用逗号分隔
        audience: 读者称呼
        scenario: 使用场景
        call_to_action: 行动口号

    Returns:
        一段完整可直接发布的夸张文案
    """
    points = [
        p.strip() for p in selling_points.replace("，", ",").split(",") if p.strip()
    ]
    point_lines = "\n".join(f"· {idx+1}）{p} ✅" for idx, p in enumerate(points))
    if not point_lines:
        point_lines = "· 太多亮点了根本写不完，自己感受！！"

    hashtags = [
        f"#{product}",
        f"# {scenario}",
        "# 必入好物",
        "# 拯救打工人",
        "# 种草不踩雷",
        "# 爆改生活",
    ]
    header = (
        f"{audience}！！！我直接破防！{scenario}被{product}狠狠拿捏，太炸裂了😭😭😭"
    )
    story = (
        f"本来只想随便试试，结果一上手就像开挂，离谱到想冲进评论区喊停！"
        f" 细节我掰开揉碎告诉你："
    )
    cta = f"{call_to_action} 不冲真的会后悔一整年！"

    return "\n".join(
        [
            header,
            "—" * 10,
            story,
            point_lines,
            "—" * 10,
            cta,
            " ".join(hashtags),
        ]
    )



万事具备，我们现在来创建我们的agent

In [ ]:
SYSTEM_PROMPT="你是一个智能的助理,你能够帮助用户来生成小红书的文案，一定要用工具的调用来生成这样的风格的文案。"

agent = create_agent(
            llm,
            system_prompt=SYSTEM_PROMPT,
            tools=[xhs_style_guide,xhs_title_pack,xhs_hype_copy],
            checkpointer=InMemorySaver(),
        )

response=agent.invoke({"messages": [{"role": "user", "content": "帮我生成一个卖衣服的小红书文案,一个百搭而且适合学生党的衣服。"}]},
                    config={
                    "configurable": {
                        "thread_id": 1,
                        "max_iterations": 10,
                    },
                    "recursion_limit": 20,
                },)
last_message=response["messages"][-1].content
print(last_message)

await service.speak(last_message)

姐妹们！！！我直接破防！校园日常上课,图书馆学习,周末逛街,约会聚餐被百搭学生党衣服狠狠拿捏，太炸裂了😭😭😭
——————————
本来只想随便试试，结果一上手就像开挂，离谱到想冲进评论区喊停！ 细节我掰开揉碎告诉你：
· 1）平价百搭 ✅
· 2）舒适透气 ✅
· 3）校园日常通勤 ✅
· 4）不挑身材 ✅
· 5）多色可选 ✅
· 6）易搭配 ✅
——————————
学生党冲鸭！马上安排！ 不冲真的会后悔一整年！
#百搭学生党衣服 # 校园日常上课,图书馆学习,周末逛街,约会聚餐 # 必入好物 # 拯救打工人 # 种草不踩雷 # 爆改生活
